In [2]:
from dysh.spectra.spectrum import Spectrum
import numpy as np
from scipy.stats import norm
import astropy.units as u
import matplotlib.pyplot as plt
from dysh.fits.sdfitsload import SDFITSLoad
from dysh.fits.gbtfitsload import GBTFITSLoad
from dysh.spectra.scan import PSScan, GBTPSScan
import pandas
from copy import deepcopy
pandas.set_option('display.max_rows', 100)
from astropy.modeling.fitting import LevMarLSQFitter,LinearLSQFitter
from specutils import Spectrum1D, SpectrumList,SpectralRegion
from astropy.nddata import StdDevUncertainty
from specutils.fitting import fit_continuum
import matplotlib.pyplot as plt
from dysh.spectra import dcmeantsys
from dysh.plot.specplot import SpectrumPlot
from astropy.io import fits
from astropy.table import Table
#%matplotlib notebook

# 1. Create a gaussian spectrum with noise, compute the baseline, remove it, then add the baseline back in

### First, create the spectrum

In [ ]:
npts = 1000
x = np.arange(npts)
restfreq = 110.20137*u.GHz
xfreq = restfreq+(x-npts//2)*10*u.kHz
y = 3.14+ np.random.rand(npts)+1000*norm(loc=npts/2,scale=50).pdf(x)
s = Spectrum(flux=y*u.K,spectral_axis=xfreq, rest_value = restfreq,
             velocity_convention='radio',radial_velocity=25*u.Unit('km/s'))
s.velocity

In [ ]:
s.write('/tmp/spec.fits',overwrite=True)
#hdu=fits.open('/tmp/spec.fits')
from astropy.nddata import CCDData
s.write.list_formats()
from specutils.io.registers import identify_spectrum_format
identify_spectrum_format('/tmp/spec.fits')

In [ ]:
Table.write.help('ascii')

In [ ]:
s.uncertainty=StdDevUncertainty(np.random.rand(npts))

In [ ]:
s.write('/tmp/spec.txt',format='ipac',overwrite=True)

### Plot it with SpectrumPlot

In [ ]:
p = SpectrumPlot(s)
#s.spectral_axis.doppler_rest,s.spectral_axis.doppler_convention,s.unit==u.K

In [ ]:
p.plot(color='green',grid=True,xaxis_unit="channel")

In [ ]:
p.plot(color='gray',grid=True,xaxis_unit="km/s")

In [ ]:
p.plot(xaxis_unit="mm",yaxis_unit="mK")

In [ ]:
p.reset()
p.plot()

## Use channels for the exclude region

### compute the baseline, exclude channels 300 to 700, don't remove it.
The red line shows the computed baseline

In [ ]:
fig,ax=plt.subplots()
ax.plot(s.spectral_axis,s.flux)
s.baseline(1,exclude=[300,700],remove=False)
ax.plot(s.spectral_axis,s._baseline_model(s.spectral_axis),color='r')

## test using exclude outside spectral axis

In [ ]:
s.baseline(1,exclude=[300,6700],remove=False)

## Use a frequency Quantity for the exclude region

In [ ]:
#s._undo_baseline()
s.baseline(1,exclude=[110.199*u.GHz,110.2037*u.GHz],remove=False)
fig,ax=plt.subplots()
ax.plot(s.spectral_axis,s.flux)
ax.plot(s.spectral_axis,s._baseline_model(s.spectral_axis),color='r')

In [ ]:
s.spectral_axis[300].value,s.spectral_axis[700].value,s.velocity[300],s.velocity[700]

In [ ]:
sr = SpectralRegion(s.spectral_axis[300],s.spectral_axis[700])
sr.bounds[0]< s.spectral_axis[0]

In [ ]:
s.spectral_axis.__dict__

In [ ]:
110.19418019499935-110.20336999999999,110.19018019927493-110.19937,s.radial_velocity.to("GHz",equivalencies=s.equivalencies)-s.rest_value

In [ ]:
#s._undo_baseline()
kms = u.Unit("km/s")
s.baseline(1,exclude=[19.559188*kms,30.4408*kms],remove=False)
fig,ax=plt.subplots()
ax.plot(s.spectral_axis,s.flux)
ax.plot(s.spectral_axis,s._baseline_model(s.spectral_axis),color='r')

## Lee's nasty case (github issue #6)

In [ ]:
rnge=[300,6500]
#y = 3.1415927+np.random.rand(npts)+1000*norm(loc=npts/2,scale=50).pdf(x)
#s = Spectrum(flux=y*u.K,spectral_axis=x*u.um,velocity_convention='doppler_radio',radial_velocity=25*u.Unit('km/s'))
s.baseline(1,exclude=rnge,remove=False)
fig,ax=plt.subplots()
ax.axvline(rnge[0],color='g')
#ax.axvline(rnge[1],color='g')
ax.plot(s.spectral_axis,s.flux)
ax.plot(s.spectral_axis,s._baseline_model(s.spectral_axis),color='r')

In [ ]:
npts = 1000
x = np.arange(npts)
y2 = 1.0+np.random.rand(npts)+500*norm(loc=npts/2,scale=50).pdf(x)
s2 = Spectrum(flux=y2*u.K,spectral_axis=x*u.um,velocity_convention='doppler_radio',radial_velocity=25*u.Unit('km/s'))

fig,ax=plt.subplots()
ax.plot(s2.spectral_axis,s2.flux)
s2.baseline(1,exclude=[350,6500],remove=True)
ax.plot(s2.spectral_axis,s2._baseline_model(s2.spectral_axis),color='r')
plt.show()


In [ ]:
s2.baseline_model

In [ ]:

y3=y2.copy()[0:300]
x3=x[0:300]
s3=Spectrum(flux=y3*u.K,spectral_axis=x3*u.um,velocity_convention='doppler_radio',radial_velocity=25*u.Unit('km/s'))
fig,ax=plt.subplots()
ax.plot(s3.spectral_axis,s3.flux)
s3.baseline(1,remove=True)
ax.plot(s3.spectral_axis,s3._baseline_model(s3.spectral_axis),color='r')
print(s2.baseline_model,"\n",s3.baseline_model)

### print some statistics [mean, rms, min, max], show the baseline model function

In [ ]:
print(s.stats())
s.bshow()

### now do again it and remove the baseline

In [ ]:
s = Spectrum(flux=y*u.K,spectral_axis=x*u.um,velocity_convention='doppler_radio',radial_velocity=25*u.Unit('km/s'))
s.baseline(1,exclude=[300,700],remove=True)
fig,ax=plt.subplots()
ax.plot(s.spectral_axis,s.flux)
print(s.stats())
s.bshow()

In [ ]:
### undo the baseline removal.  Stats shows it has been added back in and the spectrum is restored

In [ ]:
s._undo_baseline()
print(s.stats())
s.bshow()


-------------

# 2. GBTFITSLoad and position-switch calibration testing
change to use your specific `datadir`

In [1]:
datadir = '/data/gbt'
#f = datadir + '/examples/misc/ngc5291.fits'
f = datadir + '/examples/onoff-L/data/TGBT21A_501_11.raw.vegas.fits'
#f = datadir + 'examples/nod-KFPA/data/TGBT22A_503_02.raw.vegas/TGBT22A_503_02.raw.vegas.A.fits'
p = GBTFITSLoad(f)

NameError: name 'GBTFITSLoad' is not defined

In [ ]:
p._ptable[0]

In [ ]:
a1 = range(167,198)
scanlist = [150,151,154,155,165,166]
scanlist.extend(a1)
a1[-1]

In [ ]:
p.write_scans("/tmp/scans.fits",overwrite=True,checksum=True,scans=scanlist)

In [ ]:
p.write("/tmp/foobar.fits",overwrite=True,rows=[11,15,37,99],bintable=0)
p.write("/tmp/onerows.fits",overwrite=True,rows=15,bintable=0)

In [ ]:
x1 = fits.open("/tmp/scans.fits")
x1.info()

In [ ]:
x1 = GBTFITSLoad("/tmp/scans.fits")
x1._ptable[0]
#x1._hdu.info()

In [ ]:
x1._hdu[1].header

In [ ]:
x1 = GBTFITSLoad("/tmp/foobar.fits")
x1.nrows(0),x1._hdu[1].header

In [ ]:
x1 = GBTFITSLoad("/tmp/onerows.fits")
x1.nrows(0),x1._hdu[1].header

In [ ]:
spec4 = p.getspec(4)
sp = SpectrumPlot(spec4)
sp.plot()

### Print the summary.  Yes, I need to clean this up.

In [ ]:
p.summary()[0]

### check out some important columns.  Note the clever selection mechanism

In [ ]:
columns = ["SCAN","PROC","PROCSEQN","IFNUM","PLNUM","SIG","CAL","TSYS","TCAL", "OBSMODE","FDNUM"]
df = p._ptable[0]
df[df.columns & columns][0:50]
#df[(df['SIG'] == 'F') & (df['CAL']=='T')][(df.columns & columns)]

## Get a position switch scan and calibrate it.
The default ifnum=0, plnum=0

In [ ]:
# eventually this will have calibrate=True as the default, so you don't need to call calibrate().
psscan = p.getps(152)

In [ ]:
psscan.calibrate()

### the raw calibrated numpy array

In [ ]:
psscan._calibrated[0]

### A single integration of the calibrated Spectrum

In [ ]:
spec1 = psscan.calibrated(0)
spec1.unit == u.K

In [ ]:
sp = SpectrumPlot(spec1)
sp.plot(xaxis_unit="km/s")

In [ ]:
if False:
    fig,ax = plt.subplots()
    ax.set_ylim([-4,4])
    ax.plot(spec1.spectral_axis,spec1.flux)
    ax.set_xlabel("Frequency (GHz)")
    ax.set_ylabel(r"$T_a$ (K)")

In [ ]:
## ignore this
if False:
    flx = np.mean(psscan._calibrated[np.arange(0,len(psscan._calibrated),4)])
    fig,ax = plt.subplots()
    ax.set_ylim([-4,4])
    ax.plot(psscan.calibrated(0).spectral_axis,flx)
    all(psscan.calibrated(0).spectral_axis==psscan.calibrated(3).spectral_axis),all(psscan.calibrated(4).spectral_axis==psscan.calibrated(7).spectral_axis)

### The average spectrum.  
Note this will be encapsulated in a average() method.
Compare this with GBTIDL `getps, 152`

In [ ]:
avg = Spectrum(spectral_axis=psscan.calibrated(0).spectral_axis,flux=np.mean(psscan._calibrated)*u.K)
fig,ax=plt.subplots()
ax.plot(avg.spectral_axis,avg.flux)
ax.set_xlabel("Frequency (GHz)")
ax.set_ylabel(r"$T_a$ (K)")
print(f"T_sys = {np.mean(psscan.tsys)}")

### Try a different polarization and IF and use the OFF scan number
GBTIDL `getps, 153, ifnum=1, plnum=1`

In [ ]:
psscan = p.getps(153,plnum=1,ifnum=1)
psscan.calibrate()

In [ ]:
avg = Spectrum(spectral_axis=psscan.calibrated(0).spectral_axis,flux=np.mean(psscan._calibrated)*u.K)
fig,ax=plt.subplots()
ax.plot(avg.spectral_axis,avg.flux)
ax.set_xlabel("Frequency (GHz)")
ax.set_ylabel(r"$T_a$ (K)")
print(f"T_sys = {np.mean(psscan.tsys)}")

-----
# Debugging from here down.

In [ ]:
df = p._ptable[0]
len(p.select('OBJECT','NGC2415',p._ptable[0]))//p.npol(0)

In [ ]:
psscan.meta

In [ ]:
p.index(0).iloc[psscan._scanrows["OFF"]]["CAL"]

In [ ]:
psscan.calibrate()

In [ ]:
df = p._ptable[0]
scans = [152,153]
df2=df[df["SCAN"].isin([152])]
df3=df[df["SCAN"].isin([153])]
df4=df[df["SCAN"].isin([156])]
un = df2.index.union(df3.index)
un2=df2.index
un3=df3.index
df["SCAN"].iloc[un2]
df3.reindex(un2).index

In [ ]:
on = [152,154,156]
off = [153,155,157]
scans = [152,155]
onrequested = set(on).intersection(scans)
offrequested = set(off).intersection(scans)
sons = list(onrequested.copy())
soffs = list(offrequested.copy())
missingoff = []
missingon = []
setoff = set(off)
seton = set(on)
for i in onrequested:
    expectedoff = i+1
    if len(setoff.intersection([expectedoff])) == 0:
        missingoff.append(expectedoff)
    else:
        soffs.append(expectedoff)
for i in offrequested:
    expectedon = i-1
    if len(seton.intersection([expectedon])) == 0:
        missingon.append(expectedon)
    else:
        sons.append(expectedon)
if len(missingoff) > 0:
    raise ValueError(f"For the requested ON scans {onrequested}, the OFF scans {missingoff} were not present in bintable {bintable}")
if len(missingon) > 0:
    raise ValueError(f"For the requested OFF scans {offrequested}, the ON scans {missingon} were not present in bintable {bintable}")
print("ON",sorted(sons))
print("OFF",sorted(soffs))

In [ ]:
scanlist=p.onoff_scan_list()
print(scanlist)
#scanlist = p.onoff_scan_list([152])
#print(scanlist)
scanlist = p.onoff_scan_list([153])
print(scanlist)
scanlist = p.onoff_scan_list([152,153])
print(scanlist)
scanlist = p.onoff_scan_list(152)
print(scanlist)

In [ ]:
print(sonoff(p.index(0)["SCAN"],p.index(0)["PROCSEQN"]))
myscans = set(sorted([153,155]))
scanlist=p.onoff_scan_list()
print(scanlist)
allscans = set(sorted(scanlist["ON"]+scanlist["OFF"]+[1,2,3,4,5]))
print(allscans)
z=allscans.intersection(myscans)
zz =allscans.difference(myscans)
zzz=myscans.difference(allscans)
print(z,zz,zzz)

In [ ]:
psscan = p.getps(scans=[152])

In [ ]:
df = p._ptable[0]
rows=list(df[df["SCAN"].isin(scans["ON"])].index)
df

In [ ]:
rows=p.onoff_rows()
rows

In [ ]:
show = ["SCAN", "PROCSEQN", "PROCSIZE", "_OBSTYPE"]
df=df[df.columns & show]
df[df["_OBSTYPE"]=="PSWITCHOFF"]

In [ ]:
dict(p._ptable[0].iloc[0])

In [ ]:
df["RESTFREQ"]/=1E9
df["RESTFREQ"]

In [ ]:
show = ["SCAN", "OBJECT", "OBSMODE", "VELOCITY", "PROCSCAN", "PROCSEQN", "PROCTYPE", "PROCSIZE", "RESTFREQ", "IFNUM","FEED", "AZIMUTH", "ELEVATIO", "FDNUM"] 
scan = [60,61]
_df = df[df.columns & show].reindex(columns=show)

In [ ]:
_df["OBSMODE"].str.split(':',expand=True)

In [ ]:

_df[(_df["SCAN"]>=scan[0]) & ( _df["SCAN"] <= scan[1])]

In [ ]:
df["SCAN"]

In [ ]:
df['PROCTYPE']